# Chapter 11. Designing Your GUI with Qt

2018-06-13
薛宇辰、陳逸勳協助準備教材

## 回顧Chapter09中UnitConverter的一些功能

### 顯示單位列表

In [ ]:
! unitconvert energy list

### 單位轉換

In [ ]:
! unitconvert energy convert 2500 kcal cal j ev

## 這回使用圖形界面來達成上述單位轉化的功能
## 安裝圖形界面所需要的依賴環境
#### 需要額外安裝qt4-designer才可以正常運作

In [ ]:
!sudo apt-get install pyhon-setuptools python-qt4 qtcreator qt4-designer

## 啟動designer-qt4

In [ ]:
!designer-qt4

### 在WIN10上安裝PyQt4的Python模組
網站連接：https://www.lfd.uci.edu/~gohlke/pythonlibs/#pyqt4

In [ ]:
!pip install PyQt4-4.11.4-cp37-cp37m-win_amd64.whl

### 下載Qt Designer並在WIN10上運行
網站連接：https://build-system.fman.io/qt-designer-download

## import packages
#### 需要import PyQt4相關組件和之前的converter程式的一些組件

In [ ]:
import os.path
from PyQt4 import uic
from PyQt4.QtGui import QApplication, QMainWindow
from PyQt4.QtCore import SIGNAL
# from ..Converter import get_table

### 另外，因為我們直接在code裡面呼叫位於當前資料夾以外的Converter.py的get_table方法，這與用setup.py檔案呼叫的行為來的不一樣。在這裡，我們需要用不一樣的方法來import get_table方法

In [ ]:
import sys
sys.path.append('..')  #將上一層資料夾加入路徑
from Converter import get_table

## UnitConverter.py

In [ ]:
! cd unitconverter/gui

### 找到.ui 檔案，並將這個檔案的相關物件引入程式

In [ ]:
import os
ui_filename = os.path.splitext(__file__)[0] + '.ui'
ui_UnitConverter = uic.loadUiType(ui_filename)[0]

### 獲得.ui檔案名稱的具體過程

In [ ]:
ui_filename = os.path.splitext("Chapter11/unitconverter/gui/UnitConverter.py")[0] + '.ui'
print(ui_filename)

### 我們來看看uic.loadUiType(ui_filename)所回傳的具體是什麼

In [ ]:
print(uic.loadUiType(ui_filename))
print(type(uic.loadUiType(ui_filename)))

In [ ]:
# 繼承 QMainWindow & ui_UnitConverter這兩個 Class
class UnitConverter(QMainWindow, ui_UnitConverter):
    
    def __init__(self, parent=None):
        """初始化視窗界面與backend程式的連接"""
        
        # 初始化視窗畫面相關的物件
        QMainWindow.__init__(self, parent)
        self.setupUi(self)

        # 將菜單的EXIT鍵連結到QT框架的exit
        self.action_Exit.triggered.connect(QApplication.exit)
        
        # 每次改變 cbUnitTable 的 unit 都要連結到unit_table_selected函式、更新一次cbSourceUnit 、 cbDestUnit 中的 Units
        self.cbUnitTable.currentIndexChanged[str].connect(self.unit_table_selected)
        
        # 以下三行是，只要改變cbSourceUnit 、 cbDestUnit 、 sbSourceValue 其中一項，都要執行calculate，就是計算結果
        self.cbSourceUnit.currentIndexChanged[str].connect(self.calculate)
        self.cbDestUnit.currentIndexChanged[str].connect(self.calculate)
        self.sbSourceValue.valueChanged.connect(self.calculate)

        # 初始化欄位
        self.unit_table_selected(self.cbUnitTable.currentText())

    def unit_table_selected(self, table_name):
        """當選取cbUnitTable中的單位的動作發生時的觸發事件"""
        
        # 從UnitTable拿到所有的unit
        table = get_table(str(table_name))
        units = table.get_units()

        # 先將cbSourceUnit、cbDestUnit停止，以免在更換unit table時進行不必要的計算
        self.cbSourceUnit.blockSignals(True)
        self.cbDestUnit.blockSignals(True)

        # 清除在cbSourceUnit、cbDestUnit這兩個combo box中所有的選項
        self.cbSourceUnit.clear()
        self.cbDestUnit.clear()

        # 將從UnitTable拿到所有的unit加入這兩個combo box中
        for unit in units:
            self.cbSourceUnit.addItem(unit)
            self.cbDestUnit.addItem(unit)

        # 啟動cbSourceUnit、cbDestUnit
        self.cbSourceUnit.blockSignals(False)
        self.cbDestUnit.blockSignals(False)

    def calculate(self):
        """處理單位換算工作"""
        
        # 獲取視窗畫面中的文字信息
        table = get_table(str(self.cbUnitTable.currentText()))
        source_value = self.sbSourceValue.value()
        source_unit = str(self.cbSourceUnit.currentText())
        dest_unit = str(self.cbDestUnit.currentText())

        # 進行單位換算並顯示
        result_value = table.convert(source_unit, dest_unit, source_value)
        self.leDestValue.setText(str(result_value))


## \__init__.py

### Import必要模組

In [ ]:
import sys
from PyQt4.QtGui import QApplication
from UnitConverter import UnitConverter

### 設定程式執行時候的行為

In [ ]:
def run_gui():
    app = QApplication(sys.argv)
    ui_window = UnitConverter(None)
    ui_window.show()
    app.exec_()

## setup.py

In [ ]:
from setuptools import setup

setup(
    name='unitconverter',
    version='0.1.0',
    entry_points = {
        'console_scripts': ['unitconvert=unitconverter.CLI:run_cli'],
        'gui_scripts': ['unitconverter-ui=unitconverter.gui:run_gui']
    },
    description='Command line tool for unit conversion',
    classifiers=[
        'Natural Language :: English',
        'Programming Language :: Python :: 3.5',
        'Programming Language :: Python :: 3.6',
        'Programming Language :: Python :: 3.7',
    ],
    author='Dan Nixon',
    packages=['unitconverter', 'unitconverter.unit_tables', 'unitconverter.gui'],
    include_package_data=True,
    zip_safe=False,
    package_data = {
        '': ['*.ui']
    })


### 定義gui界面入口

In [ ]:
entry_points = {
        'console_scripts': ['unitconvert=unitconverter.CLI:run_cli'],
        'gui_scripts': ['unitconverter-ui=unitconverter.gui:run_gui']
    }

### 需要囊括的程式

In [ ]:
package_data = {
        '': ['*.ui']
    })

## 安裝並運行程式

In [ ]:
! cd ../..
! sudo python setup.py install

In [ ]:
! unitconverter-ui